Imported libraries and preprocessed data

In [17]:
import pandas as pd
from sklearn.model_selection import train_test_split

importData = pd.read_csv('Reviews.csv')
data = importData[['ProductId', 'Text']]
print(data.info())
print(data.head(10))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 568454 entries, 0 to 568453
Data columns (total 2 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   ProductId  568454 non-null  object
 1   Text       568454 non-null  object
dtypes: object(2)
memory usage: 8.7+ MB
None
    ProductId                                               Text
0  B001E4KFG0  I have bought several of the Vitality canned d...
1  B00813GRG4  Product arrived labeled as Jumbo Salted Peanut...
2  B000LQOCH0  This is a confection that has been around a fe...
3  B000UA0QIQ  If you are looking for the secret ingredient i...
4  B006K2ZZ7K  Great taffy at a great price.  There was a wid...
5  B006K2ZZ7K  I got a wild hair for taffy and ordered this f...
6  B006K2ZZ7K  This saltwater taffy had great flavors and was...
7  B006K2ZZ7K  This taffy is so good.  It is very soft and ch...
8  B000E7L2R4  Right now I'm mostly just sprouting this so my...
9  B00171APVA  This is a very healt

In [18]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
for x in range(0, len(data['Text']), 1000) :
    X = vectorizer.fit_transform(data['Text'])
    feature_names = vectorizer.get_feature_names_out()
    print(X)
    print(feature_names)


  (0, 55826)	2
  (0, 23609)	1
  (0, 97132)	1
  (0, 78138)	2
  (0, 107849)	2
  (0, 115220)	1
  (0, 26441)	1
  (0, 40480)	1
  (0, 48990)	1
  (0, 86858)	1
  (0, 8702)	3
  (0, 49440)	1
  (0, 107923)	1
  (0, 7659)	1
  (0, 109062)	1
  (0, 20503)	1
  (0, 52974)	1
  (0, 88452)	1
  (0, 86838)	2
  (0, 67849)	1
  (0, 73778)	1
  (0, 66909)	1
  (0, 103013)	1
  (0, 107778)	2
  (0, 86746)	1
  :	:
  (568452, 110279)	1
  (568452, 68189)	1
  (568452, 109990)	1
  (568452, 85459)	1
  (568452, 40502)	1
  (568452, 76881)	1
  (568452, 54201)	1
  (568452, 92587)	1
  (568453, 8702)	1
  (568453, 86838)	1
  (568453, 61953)	1
  (568453, 61806)	1
  (568453, 10435)	2
  (568453, 117821)	1
  (568453, 114703)	1
  (568453, 78650)	1
  (568453, 113620)	1
  (568453, 8128)	1
  (568453, 95083)	1
  (568453, 6518)	1
  (568453, 89452)	1
  (568453, 105629)	1
  (568453, 28025)	1
  (568453, 115010)	1
  (568453, 51555)	1
['00' '000' '0000' ... 'être' 'île' 'ît']
  (0, 55826)	2
  (0, 23609)	1
  (0, 97132)	1
  (0, 78138)	2
  (0, 107

KeyboardInterrupt: 